In [2]:
import pymc3 as pm
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import arviz as az

In [4]:
%config InlineBackend.figure_format = 'retina'
az.style.use('arviz-darkgrid')

How to calculate the chance of being a vampire from a test?

$$ \frac{Pr(vampire)}{Pr(mortal)} \frac{Pr(positive \mid vampire)}{Pr(positive \mid mortal)} = \frac{Pr(vampire \mid positive)}{Pr(mortal \mid positive)}$$

$$ Pr(vampire \mid positive) = \frac{Pr(vampire) Pr(positive \mid vampire)}{Pr(positive)}$$

$$ Pr(mortal \mid positive) = \frac{Pr(mortal) Pr(positive \mid mortal)}{Pr(positive)}$$

$$ Pr(positive) = Pr(positive \mid vampire) + Pr(positive \mid mortal) = \frac{Pr(positive) Pr(vampire \mid positive)}{Pr(vampire)} + \frac{Pr(positive) Pr(mortal \mid positive)}{Pr(mortal)} $$

$$ Pr(mortal) = 1 - Pr(vampire) $$